In [165]:
import pandas as pd
import json
import os
import datetime
import re
import numpy as np
from socceraction.data.statsbomb import StatsBombLoader
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import matplotsoccer as mps
import matplotlib.pyplot as plt
from socceraction.xthreat import load_model, get_successful_move_actions
from mplsoccer import VerticalPitch
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pickle 



# Ruta al archivo Excel local
file_path = r"G:\Mi unidad\TFM\data\Silva_Valladares(1).xlsx"

# Ruta al archivo Manual.json (ajústala si es necesario)
manual_path = r"G:\Mi unidad\TFM\data\Manual.json"

# Cargar el Excel
df = pd.read_excel(file_path)

# Cargar el manual
with open(manual_path, 'r', encoding='utf-8') as f:
    Manual = json.load(f)

# Procesamiento de datos
df['Categoria_Completa'] = df['Categoría'].ffill()
df['ID'] = (df['Categoría'].notna()).cumsum()
df_ordenado = df[['ID', 'Categoria_Completa', 'Descriptores', 'Inicio', 'Fin', 'Click', 'Duración']]

def encontrar_elemento(texto, lista):
    if pd.isnull(texto):
        return None
    for elemento in lista:
        # busca coincidencia exacta (word boundary)
        if re.search(rf'\b{re.escape(elemento)}\b', texto, flags=re.IGNORECASE):
            return elemento
    return None

# Función para marcar con 1 o 0
def marcar_categorizadores(texto, lista):
    texto = texto.lower() if pd.notnull(texto) else ""
    return {cat: int(cat.lower() in texto) for cat in lista}

# Traemos las listas del diccionario json
categorias = Manual['CATEGORY']
players = Manual['PLAYERS']
outcomes = Manual['OUTCOME']
zonas_golpeo = Manual['ZONA GOLPEO']
tiempos = Manual['TIEMPO']
categorizadores = Manual['CATEGORIZADORES']

# Crear lista de filas
filas_nuevas = []
for idx, grupo in df_ordenado.groupby('ID'):
    descriptores = " ".join(grupo['Descriptores'].dropna())
    fila = {
        'ID': idx,
        'CATEGORY': encontrar_elemento(grupo['Categoria_Completa'].iloc[0], categorias),
        'PLAYERS': encontrar_elemento(descriptores, players),
        'OUTCOME': encontrar_elemento(descriptores, outcomes),
        'ZONA_GOLPEO': encontrar_elemento(descriptores, zonas_golpeo),
        'TIEMPO': encontrar_elemento(descriptores, tiempos)
    }
    fila.update(marcar_categorizadores(descriptores, categorizadores))
    filas_nuevas.append(fila)

# Crear nuevo DataFrame con los eventos
df_eventos = pd.DataFrame(filas_nuevas)

# Función para tratar coordenadas
def trata_coordenadas(df):
    coordenadas = df[df.Descriptores.str.startswith("X:") | df.Descriptores.str.startswith("Y:")][['Descriptores','ID']]
    coordenadas['tipo'] = coordenadas['Descriptores'].str.extract(r'([XY]):')
    coordenadas['valor'] = coordenadas['Descriptores'].str.extract(r':(\d+)').astype(float)
    coordenadas['orden'] = coordenadas.groupby(['ID', 'tipo']).cumcount() + 1
    coordenadas = coordenadas.pivot_table(index='ID', columns=['tipo', 'orden'], values='valor')
    coordenadas.columns = [f"{col[0]}{col[1]}" for col in coordenadas.columns]
    coordenadas = coordenadas.reset_index()
    return coordenadas

# Obtener coordenadas
df_coordenadas = trata_coordenadas(df_ordenado)
df_coordenadas_seleccion = df_coordenadas[['ID', 'X1', 'X2', 'Y1', 'Y2']]

# Unir coordenadas al dataframe de eventos
df_eventos = pd.merge(df_eventos, df_coordenadas_seleccion, how='left', on='ID')

# Unir tiempos e info adicional
df_ordenado = df_ordenado.rename(columns={'Duración': 'Duracion'})
df_eventos = pd.merge(
    df_eventos,
    df_ordenado[df_ordenado['Inicio'].notna()][['ID', 'Inicio', 'Fin', 'Click', 'Duracion']],
    how='left',
    on='ID'
)

# Extraer nombre base del archivo original sin extensión
file_name = os.path.splitext(os.path.basename(file_path))[0]

# Crear un timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

# Carpeta donde se guardarán los archivos limpios
cleaned_folder = os.path.join(os.path.dirname(file_path), 'data_cleaned')
os.makedirs(cleaned_folder, exist_ok=True)

# Nombre del archivo limpio
output_file_name = f"{file_name}_cleaned_{timestamp}.csv"
output_path = os.path.join(cleaned_folder, output_file_name)

# Extraer nombre base del archivo para PARTIDO_ID (antes del primer paréntesis)
match = re.match(r"^(.*?)\(", file_name)
partido_id = match.group(1) if match else file_name

# Añadir columna PARTIDO_ID
df_eventos['PARTIDO_ID'] = partido_id

In [166]:
#Adaptación del df_eventos al formato opta 
#Adaptamos columna PARTIDO_ID a formato OPTA (Se hace de la siguiente forma silva será nºequipo 1 y valladares nºequipo 2, 2425temporada, así para cada equipo, el siguiente sera nºequipo 3...)
map_match_id = {
    'Silva_Valladares': 122425
}

# Aplicar el mapeo
df_eventos["matchId"] = df_eventos["PARTIDO_ID"].map(map_match_id)


In [167]:
from datetime import datetime

# Obtener la fecha actual en formato YYYYMMDD
fecha_actual = datetime.today().strftime('%Y%m%d')

# Crear la nueva columna 'id' en df_eventos
df_eventos['id'] = df_eventos['ID'].astype(str) + '2425' + fecha_actual


In [168]:
# Reemplazar valores en 'TIEMPO'
df_eventos['period_displayName'] = df_eventos['TIEMPO'].replace({
    '1tiempo': 'FirstHalf',
    '2 tiempo': 'SecondHalf'
})

In [169]:
# Primero, reemplazar strings vacíos por NaN para que pandas los reconozca como faltantes
df_eventos['period_displayName'].replace('', pd.NA, inplace=True)

# Ahora rellenar hacia adelante con el último valor válido
df_eventos['period_displayName'].fillna(method='ffill', inplace=True)

df_eventos['period_value'] = df_eventos['period_displayName'].map({
    'FirstHalf': 1,
    'SecondHalf': 2
}).astype('Int64')  # nullable int por si acaso queda algún NaN


C:\Users\Usuario\AppData\Local\Temp\ipykernel_24136\179542038.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_eventos['period_displayName'].replace('', pd.NA, inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_24136\179542038.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

In [170]:
# Asegurarse de que 'Click' es string
df_eventos['Click'] = df_eventos['Click'].astype(str)

# Separar en columnas de minuto y segundo
df_eventos[['minute', 'second']] = df_eventos['Click'].str.split(':', expand=True).astype(int)

# Calcular total de segundos
df_eventos['Segundos'] = df_eventos['minute'] * 60 + df_eventos['second']

In [171]:
# Invertir el diccionario: nombre ➝ ID
nombre_a_id = {v: k for k, v in jugadores_3division.items()}
# Renombrar la columna
df_eventos.rename(columns={"PLAYERS": "playerName"}, inplace=True)

# Crear columna playerId
df_eventos["playerId"] = df_eventos["playerName"].map(nombre_a_id)

In [172]:
# Mapeo de valores
mapa_body_part = {
    "PIE IZQUIERDO": "LeftFoot",
    "PIE DERECHO": "RightFoot",
    "CABEZA": "Head",
    "OTRO": "OtherBodyPart"
}

# Crear la nueva columna 'body_part'
df_eventos["body_part"] = df_eventos["ZONA_GOLPEO"].map(mapa_body_part)

# Reemplazar NaN por "Unknown"
df_eventos["body_part"] = df_eventos["body_part"].fillna("Unknown")

In [173]:
# Lista de condiciones para crear la columna 'type_displayName'
condiciones = [
    (df_eventos['SAQUE CENTRO'] == 1),
    (df_eventos['CATEGORY'] == 'PASE'),
    (df_eventos['CATEGORY'] == 'PORTERO'),
    (df_eventos['CATEGORY'] == 'RECUPERACION BALON'),
    (df_eventos['CATEGORY'] == 'REGATE'),
    (df_eventos['IN'] == 1),
    (df_eventos['OFF'] == 1),
    (df_eventos['CATEGORY'] == 'FALTA'),
    (df_eventos['GOL'] == 1),
    (df_eventos['CATEGORY'] == 'DISPUTA'),
    (df_eventos['CORNER'] == 1),
    (df_eventos['AEREO GANADO'] == 1),
    (df_eventos['CATEGORY'] == 'TIRO') & (df_eventos['POSTE'] == 1),
    (df_eventos['CATEGORY'] == 'TIRO') & (df_eventos['OFF T'] == 1),
    (df_eventos['CATEGORY'] == 'TIRO') & ((df_eventos['PARADA'] == 1) | (df_eventos['BLOQUEADO'] == 1)),
    (df_eventos['CATEGORY'] == 'BLOQUEOS DEFENSIVO') & (df_eventos['INTERCEPTADO'] == 1),
    (df_eventos['CATEGORY'] == 'BLOQUEOS DEFENSIVO') & (df_eventos['DESVIADO'] == 1),
    (df_eventos['CATEGORY'] == 'FALTA'),
]

# Valores que se asignarán en función de las condiciones
valores = [
    'Start',
    'Pass',
    'KeeperPickup',
    'BallRecovery',
    'TakeOn',
    'SubstitutionOn',
    'SubstitutionOff',
    'Foul',
    'Goal',
    'Challenge',
    'CornerAwarded',
    'Aerial',
    'Post',
    'MissedShots',
    'SavedShot',
    'BlockedPass',
    'Clearance',
    'Foul'
]

# Aplicar las condiciones y valores para crear la nueva columna
df_eventos['type_displayName'] = np.select(condiciones, valores, default=np.nan)

In [174]:

# Ruta al archivo JSON en tu ordenador (cámbiala por la ruta real)
ruta_json = r"G:\Mi unidad\TFM\scripts\jugadores_3division.json"

# Cargar el JSON
with open(ruta_json, 'r', encoding='utf-8') as f:
    players_teams = json.load(f)

# Función para mapear playerId a teamName y teamId
def map_team_info(player_id):
    info = players_teams.get(player_id, None)
    if info:
        return pd.Series([info['team'], 301 if info['team'] == 'silva' else 302])
    else:
        return pd.Series([None, None])

# Aplicar la función al df_eventos y crear columnas nuevas
df_eventos[['teamName', 'teamId']] = df_eventos['playerId'].apply(map_team_info)
df_eventos

,ID,CATEGORY,playerName,OUTCOME,ZONA_GOLPEO,TIEMPO,SAQUE CENTRO,RASO,OFENSIVA,ALTO,...,period_displayName,period_value,minute,second,Segundos,playerId,body_part,type_displayName,teamName,teamId
0,1,PASE,PARAFITA,COMPLETO,PIE IZQUIERDO,1tiempo,1,1,0,0,...,FirstHalf,1,0,1,1,16silva,LeftFoot,Start,silva,301.0
1,2,PASE,MONTERO,COMPLETO,PIE DERECHO,1tiempo,0,1,0,0,...,FirstHalf,1,0,6,6,18silva,RightFoot,Pass,silva,301.0
2,3,PASE,MARCOS,COMPLETO,PIE IZQUIERDO,1tiempo,0,1,0,0,...,FirstHalf,1,0,9,9,22silva,LeftFoot,Pass,silva,301.0
3,4,REGATE,MELO,COMPLETO,None,1tiempo,0,0,0,0,...,FirstHalf,1,0,12,12,7silva,Unknown,TakeOn,silva,301.0
4,5,FALTA,MELO,None,None,1tiempo,0,0,1,0,...,FirstHalf,1,0,17,17,7silva,Unknown,Foul,silva,301.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,517,BLOQUEOS DEFENSIVO,URIEL,None,None,2 tiempo,0,0,0,0,...,SecondHalf,2,101,55,6115,23silva,Unknown,Clearance,silva,301.0
517,518,REGATE,CARRO,INCOMPLETO,None,2 tiempo,0,0,0,0,...,SecondHalf,2,102,3,6123,10silva,Unknown,TakeOn,silva,301.0
518,519,RECUPERACION BALON,CARRO,COMPLETO,None,2 tiempo,0,0,0,0,...,SecondHalf,2,102,27,6147,10silva,Unknown,BallRecovery,silva,301.0
519,520,PASE,CARRO,COMPLETO,PIE DERECHO,2 tiempo,0,1,0,0,...,SecondHalf,2,102,28,6148,10silva,RightFoot,Pass,silva,301.0


In [175]:
# Renombrar la columna OUTCOME
df_eventos = df_eventos.rename(columns={'OUTCOME': 'outcomeType_displayName'})

# Reemplazar valores por nombres en inglés estándar
df_eventos['outcomeType_displayName'] = df_eventos['outcomeType_displayName'].replace({
    'COMPLETO': 'Successful',
    'INCOMPLETO': 'Unsuccessful'
})

# Rellenar valores vacíos con 'Successful'
df_eventos['outcomeType_displayName'] = df_eventos['outcomeType_displayName'].fillna('Successful')


In [176]:
# Crear columna outcomeType_value: 1 si Successful, 0 en caso contrario
df_eventos['outcomeType_value'] = df_eventos['outcomeType_displayName'].apply(
    lambda x: 1 if x == 'Successful' else 0
)

In [177]:
# Renombrar columnas de coordenadas al estilo Opta
df_eventos = df_eventos.rename(columns={
    'X1': 'x',
    'Y1': 'y',
    'X2': 'endX',
    'Y2': 'endY'
})

In [178]:
# Crear la columna isGoal
df_eventos['isGoal'] = df_eventos['type_displayName'].str.lower().eq('goal').astype(int)


In [179]:
# Lista de valores únicos extraídos + 'Pass'
eventos_touch = [
    'Interception', 'TakeOn', 'Tackle', 'BallTouch', 'BlockedPass',
    'Clearance', 'SavedShot', 'Save', 'OffsidePass', 'Foul', 'Dispossessed',
    'Smother', 'Claim', 'MissedShots', 'Goal', 'BlockedPass', 'Pass'
]

# Crear columna isTouch (sin distinción de mayúsculas)
df_eventos['isTouch'] = df_eventos['type_displayName'].str.lower().isin(
    [e.lower() for e in eventos_touch]
).astype(int)


In [205]:
# Preparación básica: convierte strings con ',' a float y calcula posiciones reales
def preparar_datos(df):
    # Conversión de coordenadas a float
    for col in ['x', 'y', 'endX', 'endY', 'minute', 'second']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
        else:
            df[col] = 0.0  # si no existe, se crea con 0

    # Rellenar NaNs en endX, endY
    df['endX'] = df['endX'].fillna(0)
    df['endY'] = df['endY'].fillna(0)

    # Calcular posiciones reales (en metros)
    df['x_real'] = df['x'] * 1.05
    df['y_real'] = df['y'] * 0.68
    df['endx_real'] = df['endX'] * 1.05
    df['endy_real'] = df['endY'] * 0.68

    # Rellenar NaNs por si acaso
    df[['x_real', 'y_real', 'endx_real', 'endy_real']] = df[['x_real', 'y_real', 'endx_real', 'endy_real']].fillna(0)

    # Cálculo de zonas
    df['ZonaX'] = (df['x_real'] // 13.125).astype(int)
    df['ZonaY'] = (df['y_real'] // 13.6).astype(int)
    df['Zona_end_X'] = (df['endx_real'] // 13.125).astype(int)
    df['Zona_end_Y'] = (df['endy_real'] // 13.6).astype(int)

    # Calcular tiempo en segundos
    df['Segundos'] = ((df['minute'] * 60) + df['second']).fillna(0).astype(int)
    
    if 'time_seconds' not in df.columns:
        df['time_seconds'] = (df['minute'] * 60 + df['second']).fillna(0).astype(float)
    return df

# Métricas de pase
def calcular_pases(df):
    df['LongPass'] = ((df['type_displayName'] == 'Pass') & (df['value_Longball'] == '1,0')).astype(int)
    df['ShortPass'] = ((df['type_displayName'] == 'Pass') & (df['value_Longball'].isna())).astype(int)
    
    # Key passes
    id_evento_keypasses = df.loc[df['value_Assisted'] == '1,0', 'value_RelatedEventId']
    df['KeyPasses'] = df['eventId'].isin(id_evento_keypasses).astype(int)

    # Progressive passes
    df['ProgressPasses'] = (
        (df['type_displayName'] == 'Pass') &
        (df['outcomeType_value'] == 1) &
        (df['x'] < df['endX']) &
        (
            ((df.endX - df.x) * 105 / 100 > 30) & (df.endX < 50) |
            ((df.endX - df.x) * 105 / 100 > 15) & (df.endX > 50) & (df.x <= 50) |
            ((df.endX - df.x) * 105 / 100 > 10) & (df.endX > 50) & (df.x >= 50)
        )
    ).astype(int)

    return df

# Toques en área rival y propia
def calcular_toques_area(df):
    df['TouchesRivalArea'] = (
        (~((df['type_displayName'] == 'Pass') & (df['value_KeeperThrow'] == 1))) &
        (df['type_value'].isin([2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 41, 42, 50, 52, 54, 61])) &
        (df['x'] > 83) &
        (df['endX'] < 100) &
        (df['y'] > 21.1) &
        (df['endY'] < 78.9)
    ).astype(int)

    df['TouchesOwnArea'] = (
        (~((df['type_displayName'] == 'Pass') & (df['value_KeeperThrow'] == 1))) &
        (df['type_value'].isin([2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 41, 42, 50, 52, 54, 61])) &
        (df['x'] > 0) &
        (df['endX'] < 17) &
        (df['y'] > 21.1) &
        (df['endY'] < 78.9)
    ).astype(int)

    return df

# Modelo xThreat
def aplicar_xT(df):
    df_xT = df[(df['isShot'] == True) | (df['type_displayName'] == 'Pass')].copy()

    # Formato SPADL
    df_xT.rename(columns={
        "id":"game_id", "eventId":"event_id", "teamId":"team_id",
        "x": "start_x", "y": "start_y", "endX":"end_x", "endY":"end_y",
        "period_value":"value", "period_displayName":"displayName",
        "type_value": "type_id", "type_displayName": "type_name",
        "outcomeType_value": "result_id", "outcomeType_displayName": "result_name",
        "playerId": "player_id"
    }, inplace=True)

    df_xT['type_name'] = np.where(df_xT['isShot'] == 1, 'shot', 'pass')
    df_xT['type_id'] = np.where(df_xT['isShot'] == 1, 11, 0)
    df_xT['isGoal'] = df_xT.isGoal.fillna(0)
    df_xT['result_name'] = np.where(
        (df_xT['type_name'] == 'shot') & (df_xT['isGoal'] != 1),
        'fail',
        np.where((df_xT['result_id'] != 1) & (df_xT['type_name'] == 'pass'),
                 'Unsuccessful', 'Successful')
    )
    df_xT[['end_x','end_y']] = df_xT[['end_x', 'end_y']].fillna(0)
    df_xT['start_x'] *= 1.05
    df_xT['start_y'] *= 0.68
    df_xT['end_x'] *= 1.05
    df_xT['end_y'] *= 0.68

    # Aplicar modelo
    model = load_model("https://karun.in/blog/data/open_xt_12x8_v1.json")
    df_xT = get_successful_move_actions(df_xT)
    df_xT["xT_value"] = model.rate(df_xT)

    # Unir de vuelta por índice (posición en el dataframe original)
    df['Xt_value'] = 0
    df.loc[df_xT.index, 'Xt_value'] = df_xT['xT_value']
    return df

#Tiros realizados
def add_shots_and_goals(df):
    df['Shot'] = df['isShot'].fillna(0).astype(int)
    df['Goal'] = df['isGoal'].fillna(0).astype(int)
    return df

#Regates
def add_dribbles(df):
    df['Dribble_Successful'] = (
        (df['type_displayName'] == 'TakeOn') &
        (df['outcomeType_displayName'] == 'Successful')
    ).astype(int)

    df['Dribble_Unsuccessful'] = (
        (df['type_displayName'] == 'TakeOn') &
        (df['outcomeType_displayName'] == 'Unsuccessful')
    ).astype(int)
    return df

#Acciones defensivas
def add_defensive_actions(df):
    acciones_defensivas_exitosas = [
        'Interception', 'BallRecovery', 'Tackle', 'BlockedPass',
        'Clearance', 'Aerial', 'Challenge', 'ShieldBallOpp', 'OffsideProvoked'
    ]
    df['DefensiveAction_Successful'] = (
        df['type_displayName'].isin(acciones_defensivas_exitosas) &
        (df['outcomeType_displayName'] == 'Successful')
    ).astype(int)

    df['DefensiveAction_Unsuccessful'] = (
        df['type_displayName'].isin(acciones_defensivas_exitosas) &
        (df['outcomeType_displayName'] == 'Unsuccessful')
    ).astype(int)

    df['Clearence'] = (df['type_displayName'] == 'Clearance').astype(int)

    df['Entries_Successful'] = (
        (df['type_displayName'] == 'Tackle') &
        (df['outcomeType_displayName'] == 'Successful')  # CORREGIDO
    ).astype(int)

    df['Aerial_Successful'] = (
        (df['type_displayName'] == 'Aerial') &
        (df['outcomeType_displayName'] == 'Successful')
    ).astype(int)

    df['Aerial_Unsuccessful'] = (
        (df['type_displayName'] == 'Aerial') &
        (df['outcomeType_displayName'] == 'Unsuccessful')
    ).astype(int)

    return df

#Parte del cuerpo utilizada por acciones
def add_body_part(df):
    body_parts_cols = {
        'value_HeadPass': 'Head',
        'value_LeftFoot': 'LeftFoot',
        'value_RightFoot': 'RightFoot',
        'value_OtherBodyPart': 'OtherBodyPart',
        'value_Hands': 'Hands'
    }

    def obtener_parte_cuerpo(row):
        for col, label in body_parts_cols.items():
            if pd.notna(row.get(col)) and row.get(col) != 0:
                return label
        return 'Unknown'

    df['body_part'] = df.apply(obtener_parte_cuerpo, axis=1)
    return df

#Duelos
def add_duels(df):
    duels = ['TakeOn', 'Tackle', 'Aerial', 'Challenge', 'ShieldBallOpp']
    
    df['Duels_Successful'] = (
        df['type_displayName'].isin(duels) &
        (df['outcomeType_displayName'] == 'Successful')
    ).astype(int)

    df['Duels_Unsuccessful'] = (
        df['type_displayName'].isin(duels) &
        (df['outcomeType_displayName'] == 'Unsuccessful')
    ).astype(int)

    return df

#Accion hacia una transición
def add_defensive_to_transition(df):
    acciones_defensivas_exitosas = [
        'Interception', 'BallRecovery', 'Tackle', 'BlockedPass',
        'Clearance', 'Aerial', 'Challenge', 'ShieldBallOpp', 'OffsideProvoked'
    ]

    df['DefensiveToTransition'] = 0

    for i in range(len(df) - 3):
        fila = df.iloc[i]
        if (
            fila['type_displayName'] in acciones_defensivas_exitosas and
            fila['outcomeType_displayName'] == 'Successful'
        ):
            equipo = fila['teamId']
            for j in range(1, 4):  # mirar las 3 siguientes filas
                siguiente = df.iloc[i + j]
                if siguiente['teamId'] == equipo and siguiente['type_displayName'] in ['Pass', 'TakeOn', 'Shot']:
                    df.at[i, 'DefensiveToTransition'] = 1
                    break

    return df
#eficiencia ofensiva
def calcular_eficiencia_ofensiva_presion_alta(df):
    df['HighTurnover'] = 0
    df['HighTurnoverShot'] = 0

    acciones_defensivas = ['Interception', 'BallRecovery']

    for i in range(len(df)):
        fila = df.iloc[i]

        if (
            fila['type_displayName'] in acciones_defensivas and
            fila['outcomeType_displayName'] == 'Successful' and
            fila['x'] > 50  # Campo rival
        ):
            equipo = fila['teamId']
            tiempo_inicio = fila['Segundos']
            df.at[i, 'HighTurnover'] = 1

            # Buscar si en los próximos eventos hay un tiro del mismo equipo en los siguientes 20 segundos
            for j in range(i + 1, min(i + 20, len(df))):  # Limita la búsqueda a los 20 siguientes eventos (ajustable)
                siguiente = df.iloc[j]
                if (
                    siguiente['teamId'] == equipo and
                    siguiente['isShot'] == 1 and
                    siguiente['Segundos'] - tiempo_inicio <= 20
                ):
                    df.at[i, 'HighTurnoverShot'] = 1
                    break
    return df
#Expected goals
def calcular_xG_modelo(df):
    """
    -----------
    df : pd.DataFrame
        DataFrame con eventos del partido.
    xg_model.pkl : Parámetros entrenados a partir de una temporada de la premier league.
    
    Retorna:
    --------
    df : pd.DataFrame
        DataFrame original con una columna nueva "xG" para los disparos.
    """
# =======================
# 1. Carga EL MODELO
# =======================

    with open(r"G:\Mi unidad\TFM\OPTA\Modelos\xg_model.pkl", 'rb') as f:
        model = pickle.load(f)
    
    b = model.params.values  # <- ESTO DEFINE b
    model_variables = model.model.exog_names[1:]  # omitir el intercepto
    # Diccionario para renombrar columnas
    rename_dict = {
        'type_value': 'eventId',
        'playerId': 'playerId',
        'matchId': 'matchId',
        'Shot': 'eventName',
        'teamId': 'teamId',
        'period_value': 'matchPeriod',
        'Segundos': 'eventSec',
        'id': 'id',
        'x': 'X',
        'y': 'Y'
    }

    # Copia y renombra columnas
    df_xG = df.rename(columns=rename_dict).copy()
    df_xG = df_xG[df_xG['eventName'] == 1]  # Filtrar solo disparos

    # Ajustar coordenadas al campo
    df_xG["X"] = (100 - df_xG['X']) * 105 / 100
    df_xG["Y"] = df_xG['Y'] * 68 / 100
    df_xG["C"] = abs(df_xG['Y'] - 50) * 68 / 100
    df_xG["Distance"] = np.sqrt(df_xG["X"]**2 + df_xG["C"]**2)

    # Calcular ángulo de disparo
    df_xG["Angle"] = np.where(
        np.arctan(7.32 * df_xG["X"] / (df_xG["X"]**2 + df_xG["C"]**2 - (7.32 / 2)**2)) > 0,
        np.arctan(7.32 * df_xG["X"] / (df_xG["X"]**2 + df_xG["C"]**2 - (7.32 / 2)**2)),
        np.arctan(7.32 * df_xG["X"] / (df_xG["X"]**2 + df_xG["C"]**2 - (7.32 / 2)**2)) + np.pi
    )

    # Variables para el modelo
    df_xG["X2"] = df_xG['X']**2
    df_xG["C2"] = df_xG['C']**2
    df_xG["AX"] = df_xG['Angle'] * df_xG['X']

    # Función de predicción de xG
    def calculate_xG(row):    
        bsum = b[0]  # Intercepto
        for i, v in enumerate(model_variables):
            bsum += b[i + 1] * row[v]
        xG = 1 / (1 + np.exp(-bsum))
        return xG

    # Aplicar función a los disparos
    df_xG["xG"] = df_xG.apply(calculate_xG, axis=1)

    # Unir columna xG al DataFrame original, asignar 0 donde no hay disparos
    df = df.copy()
    df = df.merge(df_xG[["id", "xG"]], on="id", how="left")
    df["xG"] = df["xG"].fillna(0)

    return df

,id,eventId,minute,second,teamId,x,y,expandedMinute,qualifiers,satisfiedEventsTypes,...,Entries_Successful,Aerial_Successful,Aerial_Unsuccessful,body_part,Duels_Successful,Duels_Unsuccessful,DefensiveToTransition,HighTurnover,HighTurnoverShot,xG
0,"2720737993,0",2,0.0,0.0,58,0.0,0.0,0,[],[],...,0,0,0,Unknown,0,0,0,0,0,0.0
1,"2720738023,0",2,0.0,0.0,62,0.0,0.0,0,[],[],...,0,0,0,Unknown,0,0,0,0,0,0.0
2,"2720738075,0",3,0.0,0.0,58,50.2,50.1,0,"[{'type': {'value': 213, 'displayName': 'Angle...","[91, 117, 30, 35, 38, 215, 218]",...,0,0,0,Unknown,0,0,0,0,0,0.0
3,"2720738095,0",4,0.0,2.0,58,28.1,42.1,0,"[{'type': {'value': 56, 'displayName': 'Zone'}...","[91, 120, 128, 36, 37, 217, 218]",...,0,0,0,Unknown,0,0,0,0,0,0.0
4,"2720738111,0",3,0.0,7.0,62,4.1,28.6,0,"[{'type': {'value': 178, 'displayName': 'Stand...",[93],...,0,0,0,Unknown,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488,"2720864009,0",865,97.0,3.0,62,0.0,0.0,103,[],[],...,0,0,0,Unknown,0,0,0,0,0,0.0
1489,"2720864103,0",800,0.0,0.0,58,0.0,0.0,13,[],[],...,0,0,0,Unknown,0,0,0,0,0,0.0
1490,"2720864089,0",866,0.0,0.0,62,0.0,0.0,13,[],[],...,0,0,0,Unknown,0,0,0,0,0,0.0
1491,"2720693817,0",1,0.0,0.0,62,0.0,0.0,0,"[{'type': {'value': 30, 'displayName': 'Involv...",[],...,0,0,0,Unknown,0,0,0,0,0,0.0
